#Árboles de Decisión

##Gráfica dispersión: Edad y Colesterol

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pacientes = pd.read_csv("https://raw.githubusercontent.com/CodigoMaquina/code/main/datos/pacientes.csv")

saludables = pacientes[pacientes["problema_cardiaco"]==0]
cardiacos = pacientes[pacientes["problema_cardiaco"]==1]

plt.figure(figsize=(6, 6))
plt.xlabel('Edad', fontsize = 20.0)
plt.ylabel('Colesterol', fontsize = 20.0)
plt.scatter(saludables["edad"], saludables["colesterol"], 
            label="Saludable (Clase: 0)", marker="*", c="skyblue", s=200)
plt.scatter(cardiacos["edad"], cardiacos["colesterol"],
            label="Cardíaco (Clase: 1)", marker="*", c="lightcoral", s=200)
plt.legend(bbox_to_anchor=(1, 0.15))
plt.show()

##Entropía

Promedio de información almacenada en una variable aleatoria (ej: lanzamiento de una moneda)

La entropia se puede calcular para diferentes bases (base 2, sistema binario). Cuanto mas bajo es el valor de entropía, es porque la probalidad de que ocurra un evento es del 100%. 

En cambio, será 1 cuando la probabilidad es del 50% en una variable binaria.

Cuanto menor sea la probabilidad de ocurrencia, menor valor de entropía.

Una entroía igual a 0 no nos sirve de nada, dado que ya sabemos de ante mano que la probabilidad de ocurriencia es del 100% para una de las dos variables.



In [4]:
from scipy.stats import entropy
from math import log

edades = pd.Series([40, 30, 20, 50])
colesterol = pd.Series([100, 110, 100, 110])

print(edades.value_counts()/edades.size) # contamos cuantas edades hay por cada valor
print(colesterol.value_counts()/colesterol.size) # contamos cuantas valores de colesterol hay por cada valor
print(entropy(edades.value_counts()/edades.size, base=2))
print(entropy(colesterol.value_counts()/colesterol.size, base=2))

40    0.25
30    0.25
20    0.25
50    0.25
dtype: float64
100    0.5
110    0.5
dtype: float64
2.0
1.0


Ejemplo de creación de arbol de decisión


In [6]:
from sklearn.model_selection import train_test_split

#asignamos datos para pruebas y para entrenamiento. El primero está compuesto por el 30%
datos_entrena, datos_prueba, clase_entrena, clase_prueba = train_test_split(
    pacientes[["edad", "colesterol"]], 
    pacientes["problema_cardiaco"], 
    test_size=0.30)

In [ ]:
from sklearn import tree #importamos arboles de decisión

arbol_decision = tree.DecisionTreeClassifier(criterion="entropy") #creamos el modelo fijando el criterio de entropía.

#al momento de crear el árbol, puedo indicar que tantas ramas quiero que me genere el árbol mediante la variable max_depth=´valor´. 
#En caso de no indicar, va a generar tantas ramas como sean necesarias para mostrar todas las instancias (generaría sobre entrenamiento)

arbol = arbol_decision.fit(datos_entrena, clase_entrena) #variable que almacena los datos del entrenamiento

accuracy = arbol_decision.score(datos_prueba, clase_prueba) # metrica que evalúa que tan bueno fue nuestro modelo con los datos de prueba

print(accuracy)

print(tree.export_text(arbol, #exporto a texto nuestro arbol
                      feature_names=["Edad", "Colesterol"])) #indico las características para que se entienda el árbol.
plt.figure(figsize=(12, 6)) 
tree.plot_tree(arbol, #genero el árbol en gráfica
              feature_names=["Edad", "Colesterol"]) #tambien es necesario pasarle las caracterísitcas para mejorar la compresión
plt.show()

##Coeficiente de GINI : impureza

Metrica que indica la probabilidad de clasificar erroneamente a una observación dado un cierto conjunto de datos.

Por cada caracteristica del set de datos, tendremos un coeficiente. Luego debemos saber cual de todas las caracteristicas es mejor para clasificar y cual peor.

 

In [13]:
import pandas as pd

# Ejemplo de datos de 10 jugadores de basket en su primer año. Toda caracterisitica tiene 2 categorias
# alto = promedio de puntos mayor que todos sus compañeros
# bajo = promedio de puntos menor que todos sus compañeros
# cada registro sigue el orden para cada grupo de datos

puntos_partido = pd.Series(["alto", "bajo", "alto", "alto", "alto",
                            "alto", "bajo", "alto", "alto", "bajo"])

minutos_partido = pd.Series(["alto", "alto", "bajo", "bajo", "bajo",
                             "alto", "bajo", "bajo", "bajo", "alto"])

rebotes_partido = pd.Series(["alto", "bajo", "bajo", "alto", "bajo",
                             "alto", "bajo", "alto", "bajo", "alto"])

asistencias_partido = pd.Series(["bajo", "bajo", "bajo", "bajo", "bajo",
                                 "bajo", "bajo", "bajo", "bajo", "bajo"])

# 1: Veterano (carrera de cinco años o más). Queremos saber si el jugador se convertirá en veterano
clase = pd.Series([1, 0, 0, 1, 0, 1, 0, 1, 0, 1])

datos = pd.DataFrame({"puntos": puntos_partido, #DF que agrupa = ´datos´
                      "minutos": minutos_partido,
                      "asistencias": asistencias_partido,
                      "rebotes": rebotes_partido,                      
                      "clase": clase})


Interpretación de la Impureza GINI

In [14]:
def impureza_gini(caracteristica, clase, datos):  #función recibe los atributos, las clases y el df 
    """str, str, DataFrame -> float"""    
    atributo_clase = datos.groupby([caracteristica, clase])[clase].count()
    atributo = datos.groupby([caracteristica])[clase].count()
    procesados = pd.merge(atributo_clase, atributo, on=[caracteristica], 
                          suffixes=('_individual', '_total')) 
    procesados["combinacion"] = (procesados[clase+"_individual"]/
                                 procesados[clase+"_total"])**2
    gini_combinacion = 1 - procesados.groupby([caracteristica, clase+"_total"])["combinacion"].sum()
    gini_pesado = (gini_combinacion * atributo) / atributo.sum() 
    return gini_pesado.sum() # obtenemos el coef


print("puntos -> impureza: %0.4f" % impureza_gini("puntos", "clase", datos))
print("minutos -> impureza: %0.4f" % impureza_gini("minutos", "clase", datos))
print("asistencias -> impureza: %0.4f" % impureza_gini("asistencias", "clase", datos))
print("rebotes -> impureza: %0.4f" % impureza_gini("rebotes", "clase", datos)) 

datos

puntos -> impureza: 0.4762
minutos -> impureza: 0.4167
asistencias -> impureza: 0.5000
rebotes -> impureza: 0.0000


,puntos,minutos,asistencias,rebotes,clase
0,alto,alto,bajo,alto,1
1,bajo,alto,bajo,bajo,0
2,alto,bajo,bajo,bajo,0
3,alto,bajo,bajo,alto,1
4,alto,bajo,bajo,bajo,0
5,alto,alto,bajo,alto,1
6,bajo,bajo,bajo,bajo,0
7,alto,bajo,bajo,alto,1
8,alto,bajo,bajo,bajo,0
9,bajo,alto,bajo,alto,1


Cada atributo posee un valor de impuereza y a su vez, todos ellos en función a la clase que me indica si el jugador va a ser veterano o no. 

Rebotes: es cero, son los mejores clasificadores. Es un clasificador perfecto, es 100% puro. Todo jugador que tenga promedio 'alto' de rebotes va a ser veterano. Si solo uso rebotes, puedo clasificar de manera perfecta a los jugadores.

Asistencias: es la mayor, es el valor máxima que se puede tomar de impureza. Los 10 jugadores tienen promedio bajo y tenemos jugadores veteranos y no. No aporta en nada para poder clasificar. Nivel de impureza alto.

Puntos y Minutos: tienen un alto nivel, aportarían algo para la clasificación objetivos.

Puntos: tienen un nivel alto de impureza, hay jugadores con promedio alto que no llegan a ser veterano y jugadores que también lo tiene y si llegan a ser veteranos.

El mejor clasificador son los rebotes, seguido de minutos y luego puntos.

**Impureza del atributo 'puntos'**

In [15]:
# Agrupo Cuántos jugadores por categoría de puntos y por clase (novato o veterano) y cuento cada clase.
puntos_y_clase = datos.groupby(["puntos", "clase"])["clase"].count()

print("\n\nJUGADORES POR PUNTOS Y CLASE\n\n", puntos_y_clase)

# Agrupo Cuántos jugadores por categoría de puntos y cuento la clase
puntos = datos.groupby(["puntos"])["clase"].count()

print("\n\nJUGADORES POR PUNTOS\n\n", puntos)

# Unir ambas series de datos para procesamiento posterior 
jugadores = pd.merge(
    puntos_y_clase,
    puntos,
    on=["puntos"], 
    suffixes=('_indivual', '_total')) 

print("\n\nUNION\n\n", jugadores)



JUGADORES POR PUNTOS Y CLASE

 puntos  clase
alto    0        3
        1        4
bajo    0        2
        1        1
Name: clase, dtype: int64


JUGADORES POR PUNTOS

 puntos
alto    7
bajo    3
Name: clase, dtype: int64


UNION

         clase_indivual  clase_total
puntos                             
alto                 3            7
alto                 4            7
bajo                 2            3
bajo                 1            3


**Cálculo de la Impureza GINI para 'puntos'**

In [12]:
# Probabilidad para cada categoría de puntos con respecto a la clase
jugadores["combinaciones"] = (jugadores["clase_indivual"]/jugadores["clase_total"])**2 #elevamos al cuadrado
print(jugadores)

# Impureza gini para cada combinación
gini_por_combinacion = 1 - jugadores.groupby(["puntos", "clase_total"])["combinaciones"].sum()
print("\n\n",gini_por_combinacion)

# Impureza gini para cada combinación con pesos
gini_con_peso_por_combinacion = (gini_por_combinacion * puntos) / puntos.sum() 
print("\n\n", gini_con_peso_por_combinacion)
print("\n", gini_con_peso_por_combinacion.sum()) #impureza de ´puntos´

        clase_indivual  clase_total  combinaciones
puntos                                            
alto                 3            7       0.183673
alto                 4            7       0.326531
bajo                 2            3       0.444444
bajo                 1            3       0.111111


 puntos  clase_total
alto    7              0.489796
bajo    3              0.444444
Name: combinaciones, dtype: float64


 puntos  clase_total
alto    7              0.342857
bajo    3              0.133333
dtype: float64

 0.4761904761904763
